# step1 - 下载图片
这里通过修改文件保存名,改为了两段式,主要是为了避免一段式命名时出现的大量重复问题

filename_parts = url.split('/')

filename = f"{filename_parts[-2]}_{filename_parts[-1]}"

In [6]:
import os
import pandas as pd
import requests
from tqdm import tqdm

def download_images(df, img_url_column, cid3_column, save_folder):

    # 获取链接列数据
    urls = df[img_url_column]

    total_images = len(df)  # 总图片数量

    # 遍历每个链接并下载内容
    with tqdm(total=total_images, desc='Downloading images', unit='image') as pbar:
        for url, cid3 in zip(urls, df[cid3_column]):
            # 在每个链接前面拼接指定的字符串
            modified_url = 'https://img20.360buyimg.com/ling/' + url
            response = requests.get(modified_url)
            if response.status_code == 200:

                # 提取文件名,这里保存的是二段式的文件名
                filename_parts = url.split('/')
                filename = f"{filename_parts[-2]}_{filename_parts[-1]}"

                # 按照 cid3 创建保存图片的文件夹（如果不存在）
                folder_path = os.path.join(save_folder, str(cid3))
                if not os.path.exists(folder_path):
                    os.makedirs(folder_path)
                    
                # 保存文件
                save_path = os.path.join(folder_path, filename)
                with open(save_path, 'wb') as f:
                    f.write(response.content)
            else:
                # print(f'{url} 下载失败')
                continue

            pbar.update(1)  # 更新进度条

    print('下载完成')

# 读取 CSV 文件
df = pd.read_csv('D://code//data//howtodo_Lv2//京喜_from_0501//京喜数据_from_0501_筛选.csv')

# 按照img_url字段进行聚合
df = df.drop_duplicates(subset=['img_url'])
# df.shape

# 在京喜项目中会出现大量的重复sku,所以这里再针对sku进行一次去重
df = df.drop_duplicates(subset=['sku'])
df.shape

# 调用函数进行图片下载
download_images(df, 'img_url', 'cid3', 'D://code//data//howtodo_Lv2//京喜_from_0501')




下载完成


# step2 - 按照新分类聚合
在现在的流程中,对cid3进行了聚合,避免太多分散的cid3,在聚合后会删除原来的cid3文件夹

In [ ]:
import pandas as pd
import os
import shutil
from tqdm import tqdm  # 导入tqdm库，用于显示进度条

# 读取CSV文件
df = pd.read_csv('D://code//data//howtodo_from_0401//服饰鞋靴箱包//品类聚类-服饰鞋靴箱包.csv')

# 处理缺失值，将缺失值填充为0
df['三级类目ID'] = df['三级类目ID'].fillna(0)

# 将浮点数转换为整数
df['三级类目ID'] = df['三级类目ID'].astype(int)

# 处理合并单元格，使用ffill()方法填充合并单元格中的值
df['聚类名称'] = df['聚类名称'].fillna(method='ffill')

# 创建目标文件夹
target_folder = 'D://code//data//howtodo_from_0401//服饰鞋靴箱包//'
os.makedirs(target_folder, exist_ok=True)

# 定义图片文件的扩展名列表
image_extensions = ['.jpg', '.jpeg', '.png', '.gif']

# 遍历每个分组
for group_name, group_df in tqdm(df.groupby('聚类名称'), desc="Processing groups"):
    
    # 创建以A列值命名的文件夹
    new_folder_path = os.path.join(target_folder, str(group_name))
    os.makedirs(new_folder_path, exist_ok=True)
    
    # 遍历B列中的文件夹名
    for folder_name in tqdm(group_df['三级类目ID'], desc="Processing folders", leave=False):
        
        # 将文件夹名转换为字符串类型
        folder_name_str = str(folder_name)
        
        # 拼接文件夹的完整路径
        folder_path = os.path.join(target_folder, folder_name_str)
        
        # 检查文件夹是否存在
        if not os.path.exists(folder_path):
            print(f"Folder '{folder_path}' does not exist. Skipping...")
            continue
        
        # 遍历文件夹中的所有文件
        for file_name in os.listdir(folder_path):
            
            # 拼接文件的完整路径
            file_path = os.path.join(folder_path, file_name)
            
            # 判断文件是否为图片文件
            if any(file_name.lower().endswith(ext) for ext in image_extensions):
                # 拼接目标文件的完整路径
                target_file_path = os.path.join(new_folder_path, file_name)
                
                # 如果目标文件已经存在，则跳过移动该文件
                if os.path.exists(target_file_path):
                    print(f"File '{target_file_path}' already exists. Skipping...")
                else:
                    # 确保目标文件夹存在
                    os.makedirs(new_folder_path, exist_ok=True)
                    
                    # 移动图片文件到新建的文件夹中
                    shutil.move(file_path, new_folder_path)

  

'''这里会删除所有为数字的文件夹'''
'''这里会删除所有为数字的文件夹'''
'''这里会删除所有为数字的文件夹'''


# 获取目标文件夹中的所有子文件夹
subfolders = [f.path for f in os.scandir(target_folder) if f.is_dir()]

# 遍历每个子文件夹
for folder in subfolders:
    # 获取子文件夹名称
    folder_name = os.path.basename(folder)
    
    # 检查子文件夹名称是否为数字
    if folder_name.isdigit():
        # 强制删除子文件夹及其内容
        shutil.rmtree(folder)
        



# step3 - 分拣包含关键词的图片
这里会先对图片进行ocr,然后将包含"京喜"关键词图片单独筛选出来


In [7]:
import os
import shutil
from paddleocr import PaddleOCR
from tqdm import tqdm

ocr = PaddleOCR(use_angle_cls=True, lang="ch", show_log=False)


def ocr_image(image_path, ocr, pbar):
    result = ocr.ocr(image_path, cls=True)

    if not result:
        pbar.write(f"Image '{os.path.basename(image_path)}' skipped due to empty OCR result.")
        pbar.update(1)
        return ""

    data_list = result[0]

    # 检查识别结果是否有关键词
    contains_keyword = False

    if data_list:
        keywords = ['京喜自营']

        # 遍历识别结果中的文本
        for data in data_list:
            text = data[1][0]  # 获取文本内容
            # 检查当前文本是否包含关键词
            if any(keyword.lower() in text.lower() for keyword in keywords):
                contains_keyword = True
                break

    if not contains_keyword:
        # pbar.write(f"Image '{os.path.basename(image_path)}' skipped due to no matching keyword.")
        pbar.update(1)
        return ""

    # 返回识别结果
    return " ".join([data[1][0] for data in data_list])


def process_folder(source_folder, destination_folder, keywords):
    # 获取所有图片文件
    image_files = []
    for root, dirs, files in os.walk(source_folder):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                image_files.append((root, file))

    # 使用tqdm创建进度条
    with tqdm(total=len(image_files), desc="Processing images") as pbar:
        for root, file in image_files:
            file_path = os.path.join(root, file)

            try:
                text = ocr_image(file_path, ocr, pbar)
                if not text:
                    continue

                # print(text)

                if any(keyword.lower() in text.lower() for keyword in keywords):
                    # 创建目标子文件夹
                    relative_path = os.path.relpath(root, source_folder)
                    new_folder = os.path.join(destination_folder, relative_path)
                    os.makedirs(new_folder, exist_ok=True)

                    # 移动文件
                    shutil.move(file_path, os.path.join(new_folder, file))
                    # print(f"Moved {file} to {new_folder}")

            except Exception as e:
                # print(f"Error processing {file}: {str(e)}")
                continue

            pbar.update(1)

# 使用示例
source_folder = "D://code//data//howtodo_Lv2//京喜_from_0501"
destination_folder = "D://code//data//howtodo_Lv2//京喜_from_0501//筛选"
keywords = ["京喜"]

process_folder(source_folder, destination_folder, keywords)




Processing images: 100%|██████████| 3352/3352 [34:37<00:00,  1.61it/s]
